<h1 align="center"> Deep Learning gender from name - RNN LSTMs </h1>

#### we will use an LSTM RNN to learn gender as f(name). we will use a stacked LSTM with many-to-one architecture feeding charecter inputs and predicting a binary outcome M/F. loss function used will be binary_crossentropy (a special case of categorical_crossentropy with m=2) and using adam optimizer (modified SGD) sample input /output would like this <br> ['r','a','k','e','s','h',' '] - male<br> ['p','r','a','d','e','e','p'] - male<br> ['g','a','n','g','a',' '] - female<br> and so on...

<img src="https://github.com/meghamattikalli/Python/blob/master/Deep%20learning%20gender/LSTM_RNN_architecture.jpg?raw=1" width="800" height="600"/>

regexp applied
[^a-zA-Z0-9 ,.\r\n] = remove
[ ]+ = ' '
[^a-zA-Z ,.\r\n] = remove
[ ]{3}+ - regex to check where 3 consecutive space occurs.

In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.


In [0]:
#parameters
maxlen = 30
labels = 2

In [4]:
from google.colab import files
uploaded = files.upload()

Saving gender_class.csv to gender_class.csv


In [0]:
input = pd.read_csv("gender_class.csv",header=None)
input.columns = ['name','m_or_f']
input['namelen']= [len(str(i)) for i in input['name']]
input1 = input[(input['namelen'] >= 2) ]

In [6]:
input1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [0]:
names = input['name']
gender = input['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
len_vocab = len(vocab)

In [8]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'7', 's', 'w', 'i', '6', 'q', 'p', 'k', 'm', 'b', 'j', '4', '9', 'v', 'e', 'g', 'u', 'r', ' ', 'z', 'f', 'END', 'a', '.', 'x', 'c', '0', 't', '2', '1', 'o', '5', '3', 'y', '8', 'd', 'h', 'l', 'n'}
vocab length is  39
length of input is  15290


In [0]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [10]:
print(char_index)

{'7': 0, 's': 1, 'w': 2, 'i': 3, '6': 4, 'q': 5, 'p': 6, 'k': 7, 'm': 8, 'b': 9, 'j': 10, '4': 11, '9': 12, 'v': 13, 'e': 14, 'g': 15, 'u': 16, 'r': 17, ' ': 18, 'z': 19, 'f': 20, 'END': 21, 'a': 22, '.': 23, 'x': 24, 'c': 25, '0': 26, 't': 27, '2': 28, '1': 29, 'o': 30, '5': 31, '3': 32, 'y': 33, '8': 34, 'd': 35, 'h': 36, 'l': 37, 'n': 38}


In [0]:
#train test split
msk = np.random.rand(len(input1)) < 0.8
train = input1[msk]
test = input1[~msk]     

In [0]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
trunc_train_name = [str(i)[0:30] for i in train.name]
for i in trunc_train_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    train_X.append(tmp)

In [13]:
np.asarray(train_X).shape

(12207, 30)

In [0]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [15]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [0]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [17]:
np.asarray(train_X).shape

(12207, 30, 39)

In [18]:
np.asarray(train_Y).shape

(12207, 2)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [19]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...


In [0]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    

In [21]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(3083, 30, 39)
(3083, 2)


In [0]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=10,validation_data=(test_X, test_Y))

Train on 12198 samples, validate on 3028 samples
Epoch 1/10
12198/12198 [==============================] - 146s - loss: 0.5867 - acc: 0.6849 - val_loss: 0.5630 - val_acc: 0.7081
Epoch 2/10
12198/12198 [==============================] - 145s - loss: 0.5312 - acc: 0.7336 - val_loss: 0.5880 - val_acc: 0.6909
Epoch 3/10
12198/12198 [==============================] - 145s - loss: 0.5217 - acc: 0.7395 - val_loss: 0.4982 - val_acc: 0.7576
Epoch 4/10
12198/12198 [==============================] - 145s - loss: 0.4866 - acc: 0.7620 - val_loss: 0.4823 - val_acc: 0.7652
Epoch 5/10
12198/12198 [==============================] - 145s - loss: 0.4682 - acc: 0.7791 - val_loss: 0.4918 - val_acc: 0.7632
Epoch 6/10
12198/12198 [==============================] - 145s - loss: 0.4583 - acc: 0.7878 - val_loss: 0.4771 - val_acc: 0.7678
Epoch 7/10
12198/12198 [==============================] - 144s - loss: 0.4525 - acc: 0.7862 - val_loss: 0.4926 - val_acc: 0.7632
Epoch 8/10
12198/12198 [========================

In [0]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3028/3028 [==============================] - 16s    
Test score: 0.453434576998
Test accuracy: 0.789299867978


In [0]:
name=["sandhya","jaspreet","rajesh"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))

In [0]:
pred

array([[ 0.62356585,  0.37643418],
       [ 0.72094178,  0.27905828],
       [ 0.90337974,  0.09662029]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [24]:
batch_size=1000
model.fit(np.array(train_X), np.array(train_Y),batch_size=batch_size,nb_epoch=50,validation_data=(np.array(test_X),np.array(test_Y)))

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 12207 samples, validate on 3083 samples
Epoch 1/50
12207/12207 [==============================] - 10s 805us/step - loss: 0.6896 - acc: 0.5581 - val_loss: 0.6835 - val_acc: 0.5540
Epoch 2/50
12207/12207 [==============================] - 7s 588us/step - loss: 0.6591 - acc: 0.6025 - val_loss: 0.6106 - val_acc: 0.6776
Epoch 3/50
12207/12207 [==============================] - 7s 588us/step - loss: 0.6123 - acc: 0.6740 - val_loss: 0.5575 - val_acc: 0.7146
Epoch 4/50
12207/12207 [==============================] - 7s 590us/step - loss: 0.5728 - acc: 0.7100 - val_loss: 0.5307 - val_acc: 0.7340
Epoch 5/50
12207/12207 [==============================] - 7s 590us/step - loss: 0.5472 - acc: 0.7309 - val_loss: 0.5209 - val_acc: 0.7554
Epoch 6/50
12207/12207 [==============================] - 7s 590us/step - loss: 0.5182 - acc: 0.7506 - val_loss: 0.5202 - val_acc: 0.7489
Epoch 7/50
12207/12207 [==============================] - 7s 590us/step - loss: 0.5052 - acc: 0.7560 - val_loss: 0.4995 - 

In [27]:
score, acc = model.evaluate(np.array(test_X), np.array(test_Y))
print('Test score:', score)
print('Test accuracy:', acc)

3083/3083 [==============================] - 5s 1ms/step
Test score: 0.327984836985645
Test accuracy: 0.8916639636717483


<h3 align="center"> lets look at the loss and accuracy chart as a function of epochs </h3><img src="https://github.com/meghamattikalli/Python/blob/master/Deep%20learning%20gender/loss_charts.bmp?raw=1" alt="loss charts" width="500" height="350"/><img src="https://github.com/meghamattikalli/Python/blob/master/Deep%20learning%20gender/acc_charts.bmp?raw=1" alt="loss charts"  width="500" height="350"/>

In [28]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.27531233, 0.7246877 ],
       [0.18733713, 0.81266284],
       [0.9639607 , 0.03603932],
       [0.04885439, 0.95114565],
       [0.00277486, 0.99722517],
       [0.9625216 , 0.03747842],
       [0.01076253, 0.9892374 ],
       [0.15015286, 0.84984714],
       [0.00635079, 0.9936492 ]], dtype=float32)

In [30]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.36553305, 0.6344669 ],
       [0.8586825 , 0.14131747],
       [0.9978422 , 0.00215775]], dtype=float32)

In [31]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.09317043, 0.9068296 ],
       [0.88855636, 0.11144372],
       [0.8764099 , 0.12359015]], dtype=float32)

In [0]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [0]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [0]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [0]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")